In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

In [3]:
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import TimeoutException

In [4]:
import os
import re
from ast import literal_eval
from konlpy.tag import Okt
from collections import Counter
from krwordrank.word import KRWordRank

In [1]:
! pip install krwordrank
! pip install JPype1>=0.7.0


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# 올리브영 리뷰 크롤링

In [38]:
# 플랫폼 통합 파일 
# 통합 코드에서는 해당 코드 삭제
korea_all = pd.read_csv('korea_all_preprocessing.csv')
korea_all.head()

,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격
0,무신사뷰티,이니스프리,화산송이 모공 바하 클렌징 폼 150g 2개,1,스킨케어,클렌징,클렌징,클렌징,4.9,7915,18200.0
1,무신사뷰티,비플레인,[대용량]녹두 약산성 클렌징 폼 160ml 2개 (+40ml 증정),2,스킨케어,클렌징,클렌징,클렌징,4.9,2870,31200.0
2,무신사뷰티,에스네이처,아쿠아 라이스 약산성 클렌징폼 160ml 2개입 [증정: 클렌징 오일 13ml + ...,3,스킨케어,클렌징,클렌징,클렌징,4.9,630,31900.0
3,무신사뷰티,토리든,[2개묶음] 다이브인 저분자 히알루론산 클렌징 폼 150ml x 2개 [총 300ml],4,스킨케어,클렌징,클렌징,클렌징,4.9,3707,23000.0
4,무신사뷰티,바닐라코,클린잇제로 오리지널 클렌징밤 대용량 180ml,5,스킨케어,클렌징,클렌징,클렌징,4.9,1215,23000.0


In [39]:
# 올리브영 크롤링 데이터만 불러오기 
# 통합 코드에서는 해당 코드 삭제
olive_top_df = korea_all[korea_all['플랫폼']=='올리브영'].copy(deep=True) 
olive_top_df.reset_index(drop=True, inplace=True) 

print(olive_top_df.info())
olive_top_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     454 non-null    object 
 1   브랜드     454 non-null    object 
 2   제품명     454 non-null    object 
 3   중분류_순위  454 non-null    int64  
 4   대분류     454 non-null    object 
 5   중분류1    454 non-null    object 
 6   중분류2    454 non-null    object 
 7   중분류3    454 non-null    object 
 8   평점      454 non-null    float64
 9   후기 수    454 non-null    int64  
 10  가격      454 non-null    float64
dtypes: float64(2), int64(2), object(7)
memory usage: 39.1+ KB
None


,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격
0,올리브영,에스트라,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,1,스킨케어,스킨케어,크림,크림,4.9,25321,59400.0
1,올리브영,에스트라,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,2,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,1957,33000.0
2,올리브영,에스트라,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,3,스킨케어,스킨케어,로션,로션,4.9,2688,53400.0
3,올리브영,바이오힐보,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml),4,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.7,2226,49500.0
4,올리브영,구달,구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획,5,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,28846,50000.0
...,...,...,...,...,...,...,...,...,...,...,...
449,올리브영,더샘,더샘 커버 퍼펙션 팁컨실러 9.5g 6colors,51,베이스메이크업,베이스메이크업,컨실러,컨실러,4.7,15210,7700.0
450,올리브영,롬앤,롬앤 듀이풀 워터 틴트,19,색조메이크업,립메이크업,립메이크업,립틴트,4.7,36721,13000.0
451,올리브영,롬앤,롬앤 한올 샤프 브로우,28,색조메이크업,아이메이크업,아이브로우,아이브로우,4.7,14002,12000.0
452,올리브영,클리오,클리오 크리스탈 글램 틴트,20,색조메이크업,립메이크업,립메이크업,립틴트,4.7,21536,18000.0


- 리뷰 크롤링 및 키워드 분석 단계
  1. 분류별 TOP5 상품 추출 -> new DF 생성
  2. 상품별 리뷰 수집(크롤링) -> 댓글 전부? => 스크롤횟수 적절히 조정해서 진행 / 리뷰 페이지 수로 결정
  3. 1 & 2 코드 통합 -> 1의 제품명이 2의 제품명에 포함되는지 여부를 기준으로 join
  4. 리뷰 키워드 분석 진행 -> 분류 / 브랜드 / 제품명 / 키워드 / 키워드별 frequency - 총 5개 컬럼 생성

## 분류별 top 상품 확인

In [ ]:
# 분류별 TOP5 상품 확인
# 1) 중분류1 고유값 확인
class_list = olive_top_df['중분류1'].unique()
class_list

array(['스킨케어', '베이스메이크업', '클렌징', '마스크팩', '선케어', '립메이크업', '아이메이크업'],
      dtype=object)

In [ ]:
# 분류별 TOP10 상품 확인
# 2) 중분류1 단위별 top10 제품만 추출

top_list = []

for i in class_list:
  top_list.append(olive_top_df[olive_top_df['중분류1'] == i][:10])

olive_top_class = pd.concat(top_list)
print(olive_top_class['중분류1'].unique())
olive_top_class

['스킨케어' '베이스메이크업' '클렌징' '마스크팩' '선케어' '립메이크업' '아이메이크업']


,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격
0,올리브영,에스트라,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,1,스킨케어,스킨케어,크림,크림,4.9,25321,59400.0
1,올리브영,에스트라,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,2,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,1957,33000.0
2,올리브영,에스트라,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,3,스킨케어,스킨케어,로션,로션,4.9,2688,53400.0
3,올리브영,바이오힐보,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml),4,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.7,2226,49500.0
4,올리브영,구달,구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획,5,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,28846,50000.0
...,...,...,...,...,...,...,...,...,...,...,...
379,올리브영,페리페라,"페리페라 올테이크 무드 팔레트 (+7,8호 컬러 마스카라 기획)",6,색조메이크업,아이메이크업,아이섀도우,아이섀도우,4.8,3509,23000.0
381,올리브영,페리페라,페리페라 스피디 스키니 브로우,7,색조메이크업,아이메이크업,아이브로우,아이브로우,4.8,58201,6000.0
382,올리브영,에뛰드,에뛰드 왓츠 인 마이 아이즈 24 colors,8,색조메이크업,아이메이크업,아이섀도우,아이섀도우,4.8,2796,6000.0
388,올리브영,투쿨포스쿨,투쿨포스쿨 스머징 트임 라이너 10colors,9,색조메이크업,아이메이크업,아이라이너,아이라이너,4.7,11257,12000.0


In [44]:
olive_top_class.head(20)

,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격
0,올리브영,에스트라,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,1,스킨케어,스킨케어,크림,크림,4.9,25321,59400.0
1,올리브영,에스트라,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,2,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,1957,33000.0
2,올리브영,에스트라,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,3,스킨케어,스킨케어,로션,로션,4.9,2688,53400.0
3,올리브영,바이오힐보,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml),4,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.7,2226,49500.0
4,올리브영,구달,구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획,5,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,28846,50000.0
5,올리브영,토리든,토리든 다이브인 저분자 히알루론산 세럼 50ml 리필기획(+리필팩 50ml),6,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.9,39708,36000.0
7,올리브영,닥터자르트,닥터자르트 시카페어 인텐시브 수딩 리페어 세럼 30ml 1+1 기획,7,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,386,49000.0
8,올리브영,차앤박,차앤박 더마앤서 액티브 부스트 PDRN앰플 30ml 기획 (+15ml 증정),8,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.9,4954,39000.0
9,올리브영,웰라쥬,웰라쥬 리얼 히알루로닉 블루 100 앰플 75ml 1+1 기획,9,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,25115,46000.0
10,올리브영,바이오힐보,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml+30ml 더블기획,11,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.7,1401,49500.0


In [24]:
# 분류별 TOP10 상품 확인
# 1) 분류별 top10 상품 추출 및 df 생성 : '스킨케어', '베이스메이크업', '클렌징', '마스크팩', '선케어', '립메이크업', '아이메이크업'
# - 스킨케어
top_list = []

top_skin = olive_top_df[olive_top_df['중분류1'] == '스킨케어'][:10]
top_skin.reset_index(drop=True, inplace=True)

top_list.append(top_skin)

top_skin.info()
print(len(top_list))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     10 non-null     object 
 1   브랜드     10 non-null     object 
 2   제품명     10 non-null     object 
 3   중분류_순위  10 non-null     int64  
 4   대분류     10 non-null     object 
 5   중분류1    10 non-null     object 
 6   중분류2    10 non-null     object 
 7   중분류3    10 non-null     object 
 8   평점      10 non-null     float64
 9   후기 수    10 non-null     int64  
 10  가격      10 non-null     float64
dtypes: float64(2), int64(2), object(7)
memory usage: 1008.0+ bytes
1


In [25]:
# 분류별 TOP10 상품 확인
# 1) 분류별 TOP10 상품 추출 및 df 생성 : '스킨케어', '베이스메이크업', '클렌징', '마스크팩', '선케어', '립메이크업', '아이메이크업'
# - 베이스메이크업 

top_base = olive_top_df[olive_top_df['중분류1'] == '베이스메이크업'][:10]
top_base.reset_index(drop=True, inplace=True)

top_list.append(top_base)

top_base.info()
print(len(top_list))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     10 non-null     object 
 1   브랜드     10 non-null     object 
 2   제품명     10 non-null     object 
 3   중분류_순위  10 non-null     int64  
 4   대분류     10 non-null     object 
 5   중분류1    10 non-null     object 
 6   중분류2    10 non-null     object 
 7   중분류3    10 non-null     object 
 8   평점      10 non-null     float64
 9   후기 수    10 non-null     int64  
 10  가격      10 non-null     float64
dtypes: float64(2), int64(2), object(7)
memory usage: 1008.0+ bytes
2


In [26]:
# 분류별 TOP10 상품 확인
# 1) 분류별 TOP10 상품 추출 및 df 생성 : '스킨케어', '베이스메이크업', '클렌징', '마스크팩', '선케어', '립메이크업', '아이메이크업'
# - 클렌징 

top_clean = olive_top_df[olive_top_df['중분류1'] == '클렌징'][:10]
top_clean.reset_index(drop=True, inplace=True)

top_list.append(top_clean)

top_clean.info()
print(len(top_list))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     10 non-null     object 
 1   브랜드     10 non-null     object 
 2   제품명     10 non-null     object 
 3   중분류_순위  10 non-null     int64  
 4   대분류     10 non-null     object 
 5   중분류1    10 non-null     object 
 6   중분류2    10 non-null     object 
 7   중분류3    10 non-null     object 
 8   평점      10 non-null     float64
 9   후기 수    10 non-null     int64  
 10  가격      10 non-null     float64
dtypes: float64(2), int64(2), object(7)
memory usage: 1008.0+ bytes
3


In [27]:
# 분류별 TOP10 상품 확인
# 1) 분류별 TOP10 상품 추출 및 df 생성 : '스킨케어', '베이스메이크업', '클렌징', '마스크팩', '선케어', '립메이크업', '아이메이크업'
# - 마스크팩

top_mask = olive_top_df[olive_top_df['중분류1'] == '마스크팩'][:10]
top_mask.reset_index(drop=True, inplace=True)

top_list.append(top_mask)

top_mask.info()
print(len(top_list))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     10 non-null     object 
 1   브랜드     10 non-null     object 
 2   제품명     10 non-null     object 
 3   중분류_순위  10 non-null     int64  
 4   대분류     10 non-null     object 
 5   중분류1    10 non-null     object 
 6   중분류2    10 non-null     object 
 7   중분류3    10 non-null     object 
 8   평점      10 non-null     float64
 9   후기 수    10 non-null     int64  
 10  가격      10 non-null     float64
dtypes: float64(2), int64(2), object(7)
memory usage: 1008.0+ bytes
4


In [28]:
# 분류별 TOP10 상품 확인
# 1) 분류별 TOP10 상품 추출 및 df 생성 : '스킨케어', '베이스메이크업', '클렌징', '마스크팩', '선케어', '립메이크업', '아이메이크업'
# - 선케어

top_sun = olive_top_df[olive_top_df['중분류1'] == '선케어'][:10]
top_sun.reset_index(drop=True, inplace=True)

top_list.append(top_sun)

top_sun.info()
print(len(top_list))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     10 non-null     object 
 1   브랜드     10 non-null     object 
 2   제품명     10 non-null     object 
 3   중분류_순위  10 non-null     int64  
 4   대분류     10 non-null     object 
 5   중분류1    10 non-null     object 
 6   중분류2    10 non-null     object 
 7   중분류3    10 non-null     object 
 8   평점      10 non-null     float64
 9   후기 수    10 non-null     int64  
 10  가격      10 non-null     float64
dtypes: float64(2), int64(2), object(7)
memory usage: 1008.0+ bytes
5


In [29]:
# 분류별 TOP10 상품 확인
# 1) 분류별 TOP10 상품 추출 및 df 생성 : '스킨케어', '베이스메이크업', '클렌징', '마스크팩', '선케어', '립메이크업', '아이메이크업'
# - 립메이크업

top_lip = olive_top_df[olive_top_df['중분류1'] == '립메이크업'][:10]
top_lip.reset_index(drop=True, inplace=True)

top_list.append(top_lip)

top_lip.info()
print(len(top_list))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     10 non-null     object 
 1   브랜드     10 non-null     object 
 2   제품명     10 non-null     object 
 3   중분류_순위  10 non-null     int64  
 4   대분류     10 non-null     object 
 5   중분류1    10 non-null     object 
 6   중분류2    10 non-null     object 
 7   중분류3    10 non-null     object 
 8   평점      10 non-null     float64
 9   후기 수    10 non-null     int64  
 10  가격      10 non-null     float64
dtypes: float64(2), int64(2), object(7)
memory usage: 1008.0+ bytes
6


In [30]:
# 분류별 TOP10 상품 확인
# 1) 분류별 TOP10 상품 추출 및 df 생성 : '스킨케어', '베이스메이크업', '클렌징', '마스크팩', '선케어', '립메이크업', '아이메이크업'
# - 아이메이크업

top_eye = olive_top_df[olive_top_df['중분류1'] == '아이메이크업'][:10]
top_eye.reset_index(drop=True, inplace=True)

top_list.append(top_eye)

top_sun.info()
print(len(top_list))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     10 non-null     object 
 1   브랜드     10 non-null     object 
 2   제품명     10 non-null     object 
 3   중분류_순위  10 non-null     int64  
 4   대분류     10 non-null     object 
 5   중분류1    10 non-null     object 
 6   중분류2    10 non-null     object 
 7   중분류3    10 non-null     object 
 8   평점      10 non-null     float64
 9   후기 수    10 non-null     int64  
 10  가격      10 non-null     float64
dtypes: float64(2), int64(2), object(7)
memory usage: 1008.0+ bytes
7


In [31]:
# 분류별 TOP10 상품 확인
# 2) 데이터셋 통합
# - reset_index(drop = False, inplace=True)
olive_top_class = pd.concat(top_list)

olive_top_class.reset_index(drop=False, inplace=True)
print(olive_top_class.info())
olive_top_class

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   70 non-null     int64  
 1   플랫폼     70 non-null     object 
 2   브랜드     70 non-null     object 
 3   제품명     70 non-null     object 
 4   중분류_순위  70 non-null     int64  
 5   대분류     70 non-null     object 
 6   중분류1    70 non-null     object 
 7   중분류2    70 non-null     object 
 8   중분류3    70 non-null     object 
 9   평점      70 non-null     float64
 10  후기 수    70 non-null     int64  
 11  가격      70 non-null     float64
dtypes: float64(2), int64(3), object(7)
memory usage: 6.7+ KB
None


,index,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격
0,0,올리브영,에스트라,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,1,스킨케어,스킨케어,크림,크림,4.9,25321,59400.0
1,1,올리브영,더마토리,더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획,1,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.6,8,26000.0
2,2,올리브영,에스트라,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,2,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,1957,33000.0
3,3,올리브영,더마토리,더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획,2,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.9,14,26000.0
4,4,올리브영,에스트라,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,3,스킨케어,스킨케어,로션,로션,4.9,2688,53400.0
...,...,...,...,...,...,...,...,...,...,...,...,...
65,5,올리브영,페리페라,"페리페라 올테이크 무드 팔레트 (+7,8호 컬러 마스카라 기획)",6,색조메이크업,아이메이크업,아이섀도우,아이섀도우,4.8,3509,23000.0
66,6,올리브영,페리페라,페리페라 스피디 스키니 브로우,7,색조메이크업,아이메이크업,아이브로우,아이브로우,4.8,58201,6000.0
67,7,올리브영,에뛰드,에뛰드 왓츠 인 마이 아이즈 24 colors,8,색조메이크업,아이메이크업,아이섀도우,아이섀도우,4.8,2796,6000.0
68,8,올리브영,투쿨포스쿨,투쿨포스쿨 스머징 트임 라이너 10colors,9,색조메이크업,아이메이크업,아이라이너,아이라이너,4.7,11257,12000.0


In [19]:
olive_top_df['브랜드'].unique()

array(['어뮤즈', '이지듀', '에스쁘아', '디어달리아', '에스트라', '에뛰드', '오브제', '식물나라', '3CE',
       '톤핏선', '투쿨포스쿨', 'VDL', '라네즈', '에이프릴스킨', '정샘물', '바닐라코', '퓌',
       '파파레서피', '더샘', '파넬', '브링그린', '네이밍', '달바', '코스노리', '쏘내추럴', '웨이크메이크',
       '차앤박', '페리페라', '루나', '지베르니', '연작', '투에이엔', '클리오', '비레디', '스킨시그널',
       '헤라', '롬앤', '듀이트리', '비디비치', '힌스', '아떼', '넘버즈인', '바이오힐보', '라운드랩',
       '토니모리', '구달', '비플레인', '닥터지', '아누아', '아이레놀', '리얼베리어', '메이크프렘',
       '스킨푸드', '베라모어', '셀퓨전씨', '프롬리에', '프리메라', '딘시', '조선미녀', '아임프롬',
       '메디큐브', 'AHC', '에스네이처', '스킨1004', '가히', '에딧비', '유이크', '더하르나이',
       '싸이닉', '아로마티카', '메디힐', '닥터자르트', '라운드어라운드', '마녀공장', '이니스프리', '토리든',
       '오드로이', '라타플랑', '로벡틴', '빌리프', '컬러그램', '하트퍼센트', '데이지크', '딘토', '투크',
       '홀리카홀리카', '엔트로피 메이크업', '뮤드', '마르시끄', '마몽드', 'SNP', '바이오던스',
       '비건이펙트', '토르홉', '아로셀', '아비브', '일소', '리터니티', '더마토리', '케어플러스',
       '하우스오브비', '얼트루', '린제이', '메디필', '메디앤서', 'CKD', '리쥬란', '한율',
       '더랩바이블랑두', '아이소이', '웰라쥬', '셀라딕스', '파티온', '반코르', '퍼셀', '제로이드',
       '온그리디

In [32]:
# 분류별 TOP5 상품 확인
# 3) 데이터셋 통합
# 중분류_순위 = index + 1

for i in range(0, len(olive_top_class)):
  olive_top_class['중분류_순위'][i] = olive_top_class['index'][i] + 1

olive_top_class.drop(columns = {'index'}, inplace=True)
olive_top_class = olive_top_class[olive_top_class['중분류_순위']<=5]
olive_top_class.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 35 entries, 0 to 64
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     35 non-null     object 
 1   브랜드     35 non-null     object 
 2   제품명     35 non-null     object 
 3   중분류_순위  35 non-null     int64  
 4   대분류     35 non-null     object 
 5   중분류1    35 non-null     object 
 6   중분류2    35 non-null     object 
 7   중분류3    35 non-null     object 
 8   평점      35 non-null     float64
 9   후기 수    35 non-null     int64  
 10  가격      35 non-null     float64
dtypes: float64(2), int64(2), object(7)
memory usage: 3.3+ KB


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16616\1275292900.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olive_top_class['중분류_순위'][i] = olive_top_class['index'][i] + 1
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16616\1275292900.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olive_top_class['중분류_순위'][i] = olive_top_class['index'][i] + 1
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16616\1275292900.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [33]:
olive_top_class

,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격
0,올리브영,에스트라,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,1,스킨케어,스킨케어,크림,크림,4.9,25321,59400.0
1,올리브영,더마토리,더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획,2,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.6,8,26000.0
2,올리브영,에스트라,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,3,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,1957,33000.0
3,올리브영,더마토리,더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획,4,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.9,14,26000.0
4,올리브영,에스트라,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,5,스킨케어,스킨케어,로션,로션,4.9,2688,53400.0
10,올리브영,어뮤즈,어뮤즈 세라믹 스킨 퍼펙터 쿠션 발레리나 에디션 한정 기획 (+리필+퍼프 증정),1,베이스메이크업,베이스메이크업,파운데이션/쿠션/BB/CC,파운데이션/쿠션/BB/CC,4.6,9122,38000.0
11,올리브영,이지듀,이지듀 기미 앰플 쿠션 15g,2,베이스메이크업,베이스메이크업,파운데이션/쿠션/BB/CC,파운데이션/쿠션/BB/CC,4.6,175,39800.0
12,올리브영,에스쁘아,에스쁘아 비벨벳 커버쿠션 SPF42 PA++ (본품+리필) 8 colors + 미니...,3,베이스메이크업,베이스메이크업,파운데이션/쿠션/BB/CC,파운데이션/쿠션/BB/CC,4.7,13489,40000.0
13,올리브영,디어달리아,디어달리아 리퀴드 블러쉬 (기획/단품),4,베이스메이크업,베이스메이크업,블러셔/쉐이딩/하이라이터,블러셔/쉐이딩/하이라이터,4.8,7588,25000.0
14,올리브영,에스트라,에스트라 아토베리어365 크림미스트 120ml,5,베이스메이크업,베이스메이크업,파우더/팩트/픽서,파우더/팩트/픽서,4.7,8269,23000.0


## 리뷰 크롤링

In [ ]:
olive_top_class['제품명'].nunique()

35

In [278]:
# 브라우저 열기
driver = webdriver.Chrome()

In [279]:
# 1. 올리브영 페이지 로드
driver.get('https://www.oliveyoung.co.kr/store/main/main.do?oy=0&utm_source=google&utm_medium=powerlink&utm_campaign=onpro_emnet_default-main_25_0101_1231&utm_content=pc_main&utm_term=%EC%98%AC%EB%A6%AC%EB%B8%8C%EC%98%81&utm_id=141045234423&gad_source=1&gclid=CjwKCAiAn9a9BhBtEiwAbKg6foao24DhngYKkXfLJpmZNukZUpPCVhDLI-MupWtyrJ2qBM3UXS7AWhoCcmIQAvD_BwE')
time.sleep(3)  # 페이지 로드 대기

In [220]:
# 경고창이 있을 때 처리하는 함수
def handle_alert(driver):
    try:
        # 경고창이 나타날 때까지 기다림
        WebDriverWait(driver, 10).until(EC.alert_is_present())
        alert = Alert(driver)
        alert.accept()  # 'OK' 버튼 클릭
        print("경고창 처리 완료")
    except:
        print("경고창 없음")

In [ ]:
'''
# 올리브영 자동 검색 반복문
wait_test = WebDriverWait(driver, 10)
search_test_list = top_class['제품명'].unique()
top_dic_test = {'제품명': []}

for idx_t, search_t in enumerate(search_test_list):
  # 검색창 클릭
  search_test_box = wait_test.until(EC.presence_of_element_located((By.XPATH, '//*[@id="query"]')))
  search_test_box.clear()
  time.sleep(1)

  # 검색어 입력 및 검색
  search_test_box.send_keys(search_t)
  search_test_box.send_keys(Keys.RETURN)
  
  # 경고창 처리
  handle_alert(driver)  

  # 새로운 페이지 링크 가져오기
  detail_tests = driver.find_elements(By.XPATH, '//*[@id="w_cate_prd_list"]/li/div/a')
  if not detail_tests:
    print(f"{idx_t}: {search_t} 상품을 찾을 수 없음")
    time.sleep(1)
    continue  # 다음 검색어로 넘어가기  
  
  # 현재 탭 저장
  current_window = driver.current_window_handle
  
  # 새로운 탭에서 상품 페이지 열기
  detail_t = detail_tests[0].get_attribute('href')
  old_tabs = driver.window_handles
  driver.execute_script(f"window.open('{detail_t}');")

  # 새로운 탭으로 전환
  wait_test.until(lambda driver: len(driver.window_handles) > len(old_tabs))
  
  new_window = [window for window in driver.window_handles if window != current_window][0]
  driver.switch_to.window(new_window)
  
  print(f"{idx_t} 새 탭으로 전환 완료!")
  
  # 새 탭에서 작업 수행 (예: 상품 정보 가져오기)
  time.sleep(3)  # 새 탭이 로드될 때까지 대기
  
  top_dic_test['제품명'].append(search_t)

  print(f"{idx_t} 상품 페이지에 접근 완료")
  
  # 작업 끝난 후 원래 탭으로 돌아오기
  driver.close()
  driver.switch_to.window(current_window)
  time.sleep(3)

print(top_dic_test)
print(len(top_dic_test['제품명']))
'''

경고창 없음
0 새 탭으로 전환 완료!
0 상품 페이지에 접근 완료
경고창 없음
1 새 탭으로 전환 완료!
1 상품 페이지에 접근 완료
경고창 없음
2 새 탭으로 전환 완료!
2 상품 페이지에 접근 완료
경고창 없음
3 새 탭으로 전환 완료!
3 상품 페이지에 접근 완료
경고창 없음
4 새 탭으로 전환 완료!
4 상품 페이지에 접근 완료
경고창 없음
5 새 탭으로 전환 완료!
5 상품 페이지에 접근 완료
경고창 없음
6 새 탭으로 전환 완료!
6 상품 페이지에 접근 완료
경고창 없음
7 새 탭으로 전환 완료!
7 상품 페이지에 접근 완료
경고창 없음
8 새 탭으로 전환 완료!
8 상품 페이지에 접근 완료
경고창 없음
9 새 탭으로 전환 완료!
9 상품 페이지에 접근 완료
경고창 없음
10 새 탭으로 전환 완료!
10 상품 페이지에 접근 완료
경고창 없음
11 새 탭으로 전환 완료!
11 상품 페이지에 접근 완료
경고창 없음
12 새 탭으로 전환 완료!
12 상품 페이지에 접근 완료
경고창 없음
13 새 탭으로 전환 완료!
13 상품 페이지에 접근 완료
경고창 없음
14 새 탭으로 전환 완료!
14 상품 페이지에 접근 완료
경고창 없음
15 새 탭으로 전환 완료!
15 상품 페이지에 접근 완료
경고창 없음
16 새 탭으로 전환 완료!
16 상품 페이지에 접근 완료
경고창 없음
17 새 탭으로 전환 완료!
17 상품 페이지에 접근 완료
경고창 없음
18 새 탭으로 전환 완료!
18 상품 페이지에 접근 완료
경고창 없음
19 새 탭으로 전환 완료!
19 상품 페이지에 접근 완료
경고창 없음
20 새 탭으로 전환 완료!
20 상품 페이지에 접근 완료
경고창 없음
21 새 탭으로 전환 완료!
21 상품 페이지에 접근 완료
경고창 없음
22 새 탭으로 전환 완료!
22 상품 페이지에 접근 완료
경고창 없음
23 새 탭으로 전환 완료!
23 상품 페이지에 접근 완료
경고창 없음
24 새 탭으로 전환 완료!
24 상품 페이지에 접근 완료
경고창 없음
25 새 탭으로 전환 완

In [ ]:
# 리뷰 페이지 크롤링 함수

def oliveyoung_reviews(driver, idx, search_p):
  wait = WebDriverWait(driver, 3)

  data = []
  count = 0
  current_page = 0
  try:
    # 리뷰 페이지로 이동 (처음 한 번만)
    review_page = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a')
    review_page.click()
    time.sleep(2)

    while count < 20:
    # -- 시작페이지 기준: count=0, current_page=0 으로 시작
    # -- 단락1 10페이지 기준: count=9, current_page=9 로 시작
    # -- 단락2 11페이지 기준: count=10, current_page=1 로 시작
    # -- 단락2 20페이지 기준: count=19, current_page=10 으로 시작

        # 리뷰 페이지 이동 및 추출
        contents = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="gdasList"]/li/div[2]/div[3]')))
        
        for content in contents:
          data.append(content.text)
        
        # 반복 횟수 확인
        count += 1 
        # -- 시작페이지 기준: count=1, current_page=0
        # -- 단락1 10페이지 기준: count=10, current_page=9
        # -- 단락2 11페이지 기준: count=11, current_page=1
        # -- 단락2 20페이지 기준: count=20, current_page=10
        
        if (count<10 and current_page<9) or (count>10 and current_page<10):
          # 다음 페이지로 넘어가기 
          next_page_no = current_page + 1
          try:
            next_page = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{next_page_no}]')))
            
            next_page.click()  
            time.sleep(3)
            current_page += 1
            # -- 시작페이지 기준: count=1, current_page=1
            # -- 단락2 11페이지 기준: count=11, current_page=2
            
          except TimeoutException: 
            print(f"TimeoutException: {idx}: {search_p}의 {next_page_no} 페이지를 찾을 수 없음")
            break
        
        else: # 다음 단락으로 넘어가기
          
          try:
            next_context = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="gdasContentsArea"]/div/div[8]/a[10]')))
            
            if count==10 and current_page==9 : # current_page=9(단락1의 10페이지일 경우)
              current_page = current_page-8
              # -- 단락1 10페이지 기준: count=10, current_page=1
              
            elif count>=10 and current_page==10 : # count >= 10 & current_page >= 10 일 경우
              current_page = current_page-9
              # -- 단락2 20페이지 기준: count=20, current_page=1
            
            next_context.click()
            time.sleep(3)
          
          except TimeoutException: 
            print(f"TimeoutException: {idx}: {search_p}의 {next_page_no} 페이지를 찾을 수 없음")
            break
        
  except Exception as e:
      print(f"{idx}: {search_p} 리뷰 수집 중 오류 발생 - {e}")

  print(f'총 {len(data)}개의 리뷰를 수집')
  print(f'총 {count}번의 반복을 수행')
  
  return data

In [ ]:
# 최종 반복문
# 올리브영 자동 검색 반복문
wait = WebDriverWait(driver, 10)
search_list = olive_top_class['제품명'].unique()
olive_top_dic = {'제품명': [], '리뷰': []}

for idx, search_p in enumerate(search_list):
  # 검색창 클릭
  search_box = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="query"]')))
  search_box.clear()
  time.sleep(1)

  # 검색어 입력 및 검색
  search_box.send_keys(search_p)
  search_box.send_keys(Keys.RETURN)
  
  # 경고창 처리
  handle_alert(driver)  

  # 새로운 페이지 링크 가져오기
  detail_pages = driver.find_elements(By.XPATH, '//*[@id="w_cate_prd_list"]/li/div/a')
  if not detail_pages:
    print(f"{idx}: {search_p} 상품을 찾을 수 없음")
    time.sleep(1)
    continue  # 다음 검색어로 넘어가기  
  
  # 현재 탭 저장
  current_window = driver.current_window_handle
  
  # 새로운 탭에서 상품 페이지 열기
  detail_page = detail_pages[0].get_attribute('href')
  old_tabs = driver.window_handles
  driver.execute_script(f"window.open('{detail_page}');")

  # 새로운 탭으로 전환
  wait.until(lambda driver: len(driver.window_handles) > len(old_tabs))
  
  new_window = [window for window in driver.window_handles if window != current_window][0]
  driver.switch_to.window(new_window)
  
  print(f"{idx} 새 탭으로 전환 완료!")
  
  # 리뷰 데이터 저장
  top_reviews = oliveyoung_reviews(driver, idx, search_p) # 리뷰 수집 함수
  
  olive_top_dic['제품명'].append(search_p)
  olive_top_dic['리뷰'].append(top_reviews)

  print(f"{idx} 상품 페이지에 접근 완료, {len(top_reviews)}개 수집 완료!")
  
  # 작업 끝난 후 원래 탭으로 돌아오기
  driver.close()
  driver.switch_to.window(current_window)
  time.sleep(3)

print(olive_top_dic)
print(len(olive_top_dic['제품명']))

경고창 없음
0 새 탭으로 전환 완료!
총 200개의 리뷰를 수집
총 20번의 반복을 수행
0 상품 페이지에 접근 완료, 200개 수집 완료!
경고창 없음
1 새 탭으로 전환 완료!
총 200개의 리뷰를 수집
총 20번의 반복을 수행
1 상품 페이지에 접근 완료, 200개 수집 완료!
경고창 없음
2 새 탭으로 전환 완료!
총 200개의 리뷰를 수집
총 20번의 반복을 수행
2 상품 페이지에 접근 완료, 200개 수집 완료!
경고창 없음
3 새 탭으로 전환 완료!
총 200개의 리뷰를 수집
총 20번의 반복을 수행
3 상품 페이지에 접근 완료, 200개 수집 완료!
경고창 없음
4 새 탭으로 전환 완료!
총 200개의 리뷰를 수집
총 20번의 반복을 수행
4 상품 페이지에 접근 완료, 200개 수집 완료!
경고창 없음
5 새 탭으로 전환 완료!
TimeoutException: 5: 에스트라 아토베리어365 크림미스트 120ml의 1 페이지를 찾을 수 없음
총 10개의 리뷰를 수집
총 1번의 반복을 수행
5 상품 페이지에 접근 완료, 10개 수집 완료!
경고창 없음
6 새 탭으로 전환 완료!
총 200개의 리뷰를 수집
총 20번의 반복을 수행
6 상품 페이지에 접근 완료, 200개 수집 완료!
경고창 없음
7 새 탭으로 전환 완료!
총 200개의 리뷰를 수집
총 20번의 반복을 수행
7 상품 페이지에 접근 완료, 200개 수집 완료!
경고창 없음
8 새 탭으로 전환 완료!
총 200개의 리뷰를 수집
총 20번의 반복을 수행
8 상품 페이지에 접근 완료, 200개 수집 완료!
경고창 없음
9 새 탭으로 전환 완료!
총 200개의 리뷰를 수집
총 20번의 반복을 수행
9 상품 페이지에 접근 완료, 200개 수집 완료!
경고창 없음
10 새 탭으로 전환 완료!
총 200개의 리뷰를 수집
총 20번의 반복을 수행
10 상품 페이지에 접근 완료, 200개 수집 완료!
경고창 없음
11 새 탭으로 전환 완료!
총 200개의 리뷰를 수집
총 20번의 반복을 수행
11 상품 

In [283]:
driver.quit()

In [ ]:
# DF 변환
olive_top_df = pd.DataFrame.from_dict(olive_top_dic, orient='index')
olive_top_df = olive_top_df.transpose()
olive_top_df.head()

,제품명,리뷰
0,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,[에스트라짱!!\n전혀 자극 없이 순하고 촉촉하고 쫀득하고 최고입니당\n가격값해요ㅠ...
1,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,[처음엔 촉촉하고 괜찮았는데 쓰다보니 좁쌀이 올라와요...모공 막는 성분이 있어서 ...
2,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,"[💙리뉴얼 전으로 돌아갔으면 좋겠어요..\n\n리뉴얼 후에도 여전히 좋긴 하지만, ..."
3,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml),[메이크업 전 화잘먹 조합 혹은 각질제거 후 관리 용으로\n제대로된 모공 제품을 사...
4,구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획,[자기전에 꼭 피부에 바르고 자는 제품입니다. 처음에 사용할 당시에는 피부가 살작 ...


In [ ]:
# 데이터셋 원본 저장
# olive_top_df.to_csv('oliveyoung_top5_reviews.csv', index=False, encoding="utf-8-sig")

# 리뷰 데이터셋 전처리

In [ ]:
# top_class 플랫폼, 중분류1, 이름 컬럼과 조인
# 1) top_cuts 형성
olive_cuts = olive_top_class[['플랫폼', '중분류1', '제품명']].copy(deep=True)
olive_cuts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35 entries, 0 to 64
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   플랫폼     35 non-null     object
 1   중분류1    35 non-null     object
 2   제품명     35 non-null     object
dtypes: object(3)
memory usage: 1.1+ KB


In [ ]:
# top_class 플랫폼, 중분류1, 이름 컬럼과 조인
# 1) top_cuts 형성
olive_top5 = pd.merge(olive_cuts, olive_top_df, how='left', on='제품명')
print(olive_top5.info())
olive_top5.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   플랫폼     35 non-null     object
 1   중분류1    35 non-null     object
 2   제품명     35 non-null     object
 3   리뷰      35 non-null     object
dtypes: object(4)
memory usage: 1.2+ KB
None


,플랫폼,중분류1,제품명,리뷰
0,올리브영,스킨케어,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,['에스트라짱!!\n전혀 자극 없이 순하고 촉촉하고 쫀득하고 최고입니당\n가격값해요...
1,올리브영,스킨케어,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,['처음엔 촉촉하고 괜찮았는데 쓰다보니 좁쌀이 올라와요...모공 막는 성분이 있어서...
2,올리브영,스킨케어,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,"['💙리뉴얼 전으로 돌아갔으면 좋겠어요..\n\n리뉴얼 후에도 여전히 좋긴 하지만,..."
3,올리브영,스킨케어,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml),['메이크업 전 화잘먹 조합 혹은 각질제거 후 관리 용으로\n제대로된 모공 제품을 ...
4,올리브영,스킨케어,구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획,['자기전에 꼭 피부에 바르고 자는 제품입니다. 처음에 사용할 당시에는 피부가 살작...


In [ ]:
olive_top5['리뷰'][0]

'[\'에스트라짱!!\\n전혀 자극 없이 순하고 촉촉하고 쫀득하고 최고입니당\\n가격값해요ㅠ\', \'올리브영에서 재구매템이 별로 없는데\\n이 제품은 재구매템에 들어가는 제품이에요 **\\n\\n일단 성분 자체가 전부 다 순해서\\n엄청난 민감성 피부도 무조건 사용하 실 수 있어요\\n갓난 애기도 사용하 수 있을 정도로 순한 것 같아요!\\n\\n그리고 극건성인 분들에게 추천합니다 !!!\\n지성인 분들에겐 조금만 발라도\\n기름이 엄청 빠르게 올라오더라구요\\n극건서인 분들이 사용하시면 아주 잘 맞으실거에요^_^\\n\\n하지만 가격이 다른 보습크림들 보다 조금 높아서\\n세일할 때 사두면 좋을 것 같아용\\n\\n재구매 의사 ㅇㅇ\', \'에스트라 아토베리어365크림 한정기획 사용후기!!\\n겨울에 사용하기 좋은 크림이예요 겨울에 건조하고 트러블올라올때 꼭 사용하는 에스트라 크림입니다!! 캡슐이 들어있고 속건조까지 잡아 주는 크림이예요 한가지 단점이라면... 너무 비싸다는거 ㅜㅜ 비싸서 부담스러운 크림이지만 겨울 크림으로 딱이예요~!!\', \'누가 언제 어디서 사용하더라도 잘 맞을 만능 보습 크림입니당. 많이 촉촉한 편이고 살짝은 미끄덩 한 제품이라 낮에는 잘 안쓰고 자기전에 스킨케어 마지막 단계에 듬뿍 발라주고 있어요.\\n\\n에스트라 아토베리어 말모.. 건성이라면 사세요!!!\', \'에스트라 아토베리어365크림 한정기획 사용후기\\n겨울크림으로 제일 좋아요 캡슐이 들어있는 아토베리어 365크림추친합니다 보습감도 좋고 속건조를 잡아줘요!! 그리고 겨울에 급 피부 뒤집어질때 이크림 쓰면 바로 피부가 좋아져서 믿고쓰는 에스트라 아토베리어 크림입니다! 다만 가격이 너무 비싸서 듬뿍듬뿍 쓸수가없어요ㅜㅜ 할인할때 사요ㅠ\', \'진짜 개 좋아요… 정착 잘 안하는데 겨울철 보습크림은 얘로 확정!! 다음 해에 구매하러 또 올게여\', \'건성에 좋고 순한 크림이라고 추천받아서 샀는데 이 크림 쓰자마자 피부가 다 뒤집어졌어요. 살면서 이마 제외하고는 여드름 나본

# 키워드 형태소 분석

## 문자열 정리

In [ ]:
olive_top5['리뷰'] = olive_top5['리뷰'].str.replace(r'\n', ' ')
olive_top5['리뷰'][0]

'[\'에스트라짱!! 전혀 자극 없이 순하고 촉촉하고 쫀득하고 최고입니당 가격값해요ㅠ\', \'올리브영에서 재구매템이 별로 없는데 이 제품은 재구매템에 들어가는 제품이에요 **  일단 성분 자체가 전부 다 순해서 엄청난 민감성 피부도 무조건 사용하 실 수 있어요 갓난 애기도 사용하 수 있을 정도로 순한 것 같아요!  그리고 극건성인 분들에게 추천합니다 !!! 지성인 분들에겐 조금만 발라도 기름이 엄청 빠르게 올라오더라구요 극건서인 분들이 사용하시면 아주 잘 맞으실거에요^_^  하지만 가격이 다른 보습크림들 보다 조금 높아서 세일할 때 사두면 좋을 것 같아용  재구매 의사 ㅇㅇ\', \'에스트라 아토베리어365크림 한정기획 사용후기!! 겨울에 사용하기 좋은 크림이예요 겨울에 건조하고 트러블올라올때 꼭 사용하는 에스트라 크림입니다!! 캡슐이 들어있고 속건조까지 잡아 주는 크림이예요 한가지 단점이라면... 너무 비싸다는거 ㅜㅜ 비싸서 부담스러운 크림이지만 겨울 크림으로 딱이예요~!!\', \'누가 언제 어디서 사용하더라도 잘 맞을 만능 보습 크림입니당. 많이 촉촉한 편이고 살짝은 미끄덩 한 제품이라 낮에는 잘 안쓰고 자기전에 스킨케어 마지막 단계에 듬뿍 발라주고 있어요.  에스트라 아토베리어 말모.. 건성이라면 사세요!!!\', \'에스트라 아토베리어365크림 한정기획 사용후기 겨울크림으로 제일 좋아요 캡슐이 들어있는 아토베리어 365크림추친합니다 보습감도 좋고 속건조를 잡아줘요!! 그리고 겨울에 급 피부 뒤집어질때 이크림 쓰면 바로 피부가 좋아져서 믿고쓰는 에스트라 아토베리어 크림입니다! 다만 가격이 너무 비싸서 듬뿍듬뿍 쓸수가없어요ㅜㅜ 할인할때 사요ㅠ\', \'진짜 개 좋아요… 정착 잘 안하는데 겨울철 보습크림은 얘로 확정!! 다음 해에 구매하러 또 올게여\', \'건성에 좋고 순한 크림이라고 추천받아서 샀는데 이 크림 쓰자마자 피부가 다 뒤집어졌어요. 살면서 이마 제외하고는 여드름 나본적이 딱히 없었는데 이거 쓰자마자 볼이며 턱이며... 여드름으로 얼굴이 뒤덮

In [ ]:
# 1. 문자열 리뷰를 리스트로 변환

def str_to_list(x):
  try:
    if type(x) == str:
      return literal_eval(x)
    elif type(x) == list:
      return x
  except:
    return None

olive_top5['리뷰'] = olive_top5['리뷰'].apply(lambda x: str_to_list(x))

olive_top5['리뷰'][0]

['에스트라짱!! 전혀 자극 없이 순하고 촉촉하고 쫀득하고 최고입니당 가격값해요ㅠ',
 '올리브영에서 재구매템이 별로 없는데 이 제품은 재구매템에 들어가는 제품이에요 **  일단 성분 자체가 전부 다 순해서 엄청난 민감성 피부도 무조건 사용하 실 수 있어요 갓난 애기도 사용하 수 있을 정도로 순한 것 같아요!  그리고 극건성인 분들에게 추천합니다 !!! 지성인 분들에겐 조금만 발라도 기름이 엄청 빠르게 올라오더라구요 극건서인 분들이 사용하시면 아주 잘 맞으실거에요^_^  하지만 가격이 다른 보습크림들 보다 조금 높아서 세일할 때 사두면 좋을 것 같아용  재구매 의사 ㅇㅇ',
 '에스트라 아토베리어365크림 한정기획 사용후기!! 겨울에 사용하기 좋은 크림이예요 겨울에 건조하고 트러블올라올때 꼭 사용하는 에스트라 크림입니다!! 캡슐이 들어있고 속건조까지 잡아 주는 크림이예요 한가지 단점이라면... 너무 비싸다는거 ㅜㅜ 비싸서 부담스러운 크림이지만 겨울 크림으로 딱이예요~!!',
 '누가 언제 어디서 사용하더라도 잘 맞을 만능 보습 크림입니당. 많이 촉촉한 편이고 살짝은 미끄덩 한 제품이라 낮에는 잘 안쓰고 자기전에 스킨케어 마지막 단계에 듬뿍 발라주고 있어요.  에스트라 아토베리어 말모.. 건성이라면 사세요!!!',
 '에스트라 아토베리어365크림 한정기획 사용후기 겨울크림으로 제일 좋아요 캡슐이 들어있는 아토베리어 365크림추친합니다 보습감도 좋고 속건조를 잡아줘요!! 그리고 겨울에 급 피부 뒤집어질때 이크림 쓰면 바로 피부가 좋아져서 믿고쓰는 에스트라 아토베리어 크림입니다! 다만 가격이 너무 비싸서 듬뿍듬뿍 쓸수가없어요ㅜㅜ 할인할때 사요ㅠ',
 '진짜 개 좋아요… 정착 잘 안하는데 겨울철 보습크림은 얘로 확정!! 다음 해에 구매하러 또 올게여',
 '건성에 좋고 순한 크림이라고 추천받아서 샀는데 이 크림 쓰자마자 피부가 다 뒤집어졌어요. 살면서 이마 제외하고는 여드름 나본적이 딱히 없었는데 이거 쓰자마자 볼이며 턱이며... 여드름으로 얼굴이 뒤덮혀버려서 깜짝 

In [ ]:
# 2. 리스트 내 문자열 전처리 : 특수문자 제거
for i in range(len(olive_top5)):
  text = olive_top5['리뷰'][i]
  
  for j in range(len(text)):
    line = text[j]
    line = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]', '', line).strip()
    olive_top5['리뷰'][i][j] = line

olive_top5['리뷰'][0][0]

'에스트라짱 전혀 자극 없이 순하고 촉촉하고 쫀득하고 최고입니당 가격값해요ㅠ'

In [ ]:
# 3. 변환값 저장할 컬럼 생성 
olive_top5['reply_text'] = ''
olive_top5['sentences_tag'] = ''

olive_top5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   플랫폼            35 non-null     object
 1   중분류1           35 non-null     object
 2   제품명            35 non-null     object
 3   리뷰             35 non-null     object
 4   reply_text     35 non-null     object
 5   sentences_tag  35 non-null     object
dtypes: object(6)
memory usage: 1.8+ KB


In [ ]:
# 4. 문자열 형태소 변환값 저장 
okt = Okt()

for i in range(len(olive_top5)):
  text = olive_top5['리뷰'][i]
  
  reply_text = []
  for line in text:
    reply_text.append(line[:-1])
  
  olive_top5['reply_text'][i] = reply_text
    
  sentences_tag = []
  for stn in reply_text:
    morph = okt.pos(stn, stem=True)
    sentences_tag.append(morph)
  
  olive_top5['sentences_tag'][i] = sentences_tag
     
olive_top5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   플랫폼            35 non-null     object
 1   중분류1           35 non-null     object
 2   제품명            35 non-null     object
 3   리뷰             35 non-null     object
 4   reply_text     35 non-null     object
 5   sentences_tag  35 non-null     object
dtypes: object(6)
memory usage: 1.8+ KB


In [ ]:
# 5-1. 형태소 저장 컬럼 생성 
olive_top5['bucket_list'] = ''
olive_top5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   플랫폼            35 non-null     object
 1   중분류1           35 non-null     object
 2   제품명            35 non-null     object
 3   리뷰             35 non-null     object
 4   reply_text     35 non-null     object
 5   sentences_tag  35 non-null     object
 6   bucket_list    35 non-null     object
dtypes: object(7)
memory usage: 2.0+ KB


In [ ]:
# 1. 형태소 분석기 초기화
okt = Okt()

# 정력 -> 세정력

'''# 2. 변환할 단어 매핑
word_mapping = {
    "좋아요": "좋음", "좋은": "좋음",
    "좋습니다": "좋음","좋네요":"좋음",
    "좋고": "좋음","좋아서":"좋음",
    "발":"발색", "오브":"오브제",
    "발림":"발림성","성도":"발림성",
    "마음":"마음에 들다","자연":"자연스러움",
    "자연스럽고":"자연스러움","자연스러운":"자연스러움",
    "색도":"색","색깔":"색",
    "색상":"색","맘":"마음에 들다",
    "부담":"부담스럽지 않은","가성":"가성비",
    "적당히":"적당한","자연스러워서": "자연스러움",
    "적당하고":"적당한","쟁":"쟁여두고 싶은",
    "옷": "옷에 묻음"}'''
    
# 3. 제거할 단어 리스트
remove_words = {"거", "구매", "유","때", "제", "입니다", "후기", "달", "것","보기","과과","중","다른"
                "같습니다","개","더","비", "입니당", "있습니다", "있고", "있어서", "있는", "있지만",
                "개", "좀", "생각","있어서","수","조금","일리","데","이","안","일단","정말","아주","사서","제일","쓰기","과",
                 "있어요","진짜","하나","있는","원래","다음","처음", "티", "저", "벌써", "상품","구입","일","좋음","좋아요","좋고","좋아서",
                 "좋습니다", "좋네요", "좋은", "배송","같아요", "가격","성도","매번","한번","원","번","이번","요","좋","같아서","보고"
                 ,"같네요","매우","듯",
                 '가', '나', '다', '라', '마', '바', '사', '아', '자', '차', '카', '타', '파', '하',
                 '에', '이', '오', '우', 
                 '사용', '제품', '상품', '에스트', '이다', '있다', '있고', '같다', '없다', '없고', '화장', '하다', '하고'
                 }

In [ ]:
# 5-2. 문장별 명사 및 형용사 단어만 남기기 
for i in range(len(olive_top5)):
  sentences_tag = olive_top5['sentences_tag'][i]
  
  bucket_list = []
  
  for one_stn in sentences_tag:
    for word, tag in one_stn:
      if tag in ['Noun', 'Adjective']:
        if word in remove_words:
          continue
        # word = word_mapping.get(word, word)
        bucket_list.append(word)
        
  olive_top5['bucket_list'][i] = bucket_list
        
print(olive_top5['bucket_list'][0])
print(olive_top5.info())

['에스', '트', '짱', '전혀', '자극', '순하다', '촉촉하다', '쫀득', '최고', '이다', '값해', '올리브영', '템', '별로', '템', '성분', '자체', '전부', '순하다', '엄청나다', '민감', '피부', '무조건', '실', '갓난', '애기', '정도', '순하다', '건성', '분', '추천', '지성인', '분', '발라', '기름', '빠르다', '건', '서인', '분', '다른', '보습', '크림', '높다', '사두', '좋다', '의사', '아토', '베리', '크림', '한정', '기획', '겨울', '좋다', '크림', '겨울', '건조하다', '트러블', '꼭', '크림', '이다', '캡슐', '건조', '크림', '한가지', '단점', '라면', '비싸다', '비싸다', '부담스럽다', '크림', '겨울', '크림', '이예', '누가', '언제', '만능', '보습', '크림', '이다', '촉촉하다', '편이', '살짝', '밉다', '덩', '낮', '스킨', '케어', '마지막', '단계', '듬뿍', '아토', '베리', '말모', '건성', '라면', '아토', '베리', '크림', '한정', '기획', '겨울', '크림', '좋다', '캡슐', '아토', '베리', '크림', '추', '친하다', '보습', '좋다', '건조', '겨울', '급', '피부', '크림', '바로', '피부', '아토', '베리', '크림', '이다', '다만', '비싸다', '할인', '좋다', '정착', '겨울철', '보습', '크림', '얘', '확정', '해', '또', '건성', '좋다', '순하다', '크림', '추천', '크림', '피부', '이마', '제외', '여드름', '나본', '딱하다', '볼', '턱', '여드름', '얼굴', '깜짝', '도저히', '안되다', '크림', '수딩젤', '바로', '피부', '완전하다', '원상', '복구', '일주일', '정도', '피부', '예민

## 형태소 변환 후 전처리

In [ ]:
# 원본 복제
olive_top5_df = olive_top5.copy(deep=True)

In [ ]:
# 6. 필요없는 컬럼 지우기
olive_top5_df.drop(['제품명', '리뷰', 'reply_text', 'sentences_tag'], axis=1, inplace=True)
olive_top5_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   플랫폼          35 non-null     object
 1   중분류1         35 non-null     object
 2   bucket_list  35 non-null     object
dtypes: object(3)
memory usage: 968.0+ bytes


In [ ]:
olive_top5_df[-5:]

,플랫폼,중분류1,bucket_list
30,올리브영,아이메이크업,"[왜, 기존, 트, 라떼, 단종, 에스쁘, 사실, 브랜드, 리뉴, 단종, 부활, 말..."
31,올리브영,아이메이크업,"[애굣살, 그릴, 좋다, 붓펜, 팬슬, 잘맞다, 습, 캬, 전, 무쌍, 라인, 진하..."
32,올리브영,아이메이크업,"[도화, 밤, 증정, 무조건, 투, 도화살, 몬안, 증정, 벚꽃, 완전, 뽀용뽀용,..."
33,올리브영,아이메이크업,"[애교살, 바로, 애교살, 필수, 템, 좋다, 다만, 지속, 살짝, 쿵, 아쉽다, ..."
34,올리브영,아이메이크업,"[지속, 짱짱하다, 눈, 시림, 번짐, 마스카라, 몇, 년, 컬, 픽스, 쟁여두, ..."


In [ ]:
# 병합 전 길이
print(len(olive_top5['bucket_list'][30]))
print(len(olive_top5['bucket_list'][31]))
print(len(olive_top5['bucket_list'][32]))
print(len(olive_top5['bucket_list'][33]))
print(len(olive_top5['bucket_list'][34]))

4654
1635
4684
4002
3723


In [ ]:
# 중분류1 단위로 bucket_list 합치기
olive_class_list = olive_top5_df['중분류1'].unique()

for j in range(len(olive_class_list)):
  bucket = olive_top5_df[olive_top5_df['중분류1']==olive_class_list[j]].index
  min_b = min(bucket)
  
  for b in bucket:
    if b > min_b:
      min_bucket = olive_top5_df['bucket_list'][min_b]      
      min_bucket += olive_top5_df['bucket_list'][b]
  
print(len(min_bucket))

18698


In [ ]:
# 병합 후 길이
print(len(olive_top5_df['bucket_list'][30]))
print(len(olive_top5_df['bucket_list'][31]))
print(len(olive_top5_df['bucket_list'][32]))
print(len(olive_top5_df['bucket_list'][33]))
print(len(olive_top5_df['bucket_list'][34]))

18698
1635
4684
4002
3723


In [ ]:
# 중분류1 단위로 한 행씩만 남기기
olive_class_list = olive_top5_df['중분류1'].unique()

for j in range(len(olive_class_list)):
  bucket = olive_top5_df[olive_top5_df['중분류1']==olive_class_list[j]].index
  min_b = min(bucket)
  max_b = max(bucket)
  
  olive_top5_df.drop(labels=range(min_b+1,max_b+1), axis=0, inplace=True)

olive_top5_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 0 to 30
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   플랫폼          7 non-null      object
 1   중분류1         7 non-null      object
 2   bucket_list  7 non-null      object
dtypes: object(3)
memory usage: 224.0+ bytes


In [ ]:
olive_top5_df.reset_index(drop=True, inplace=True)
olive_top5_df

,플랫폼,중분류1,bucket_list
0,올리브영,스킨케어,"[에스, 트, 짱, 전혀, 자극, 순하다, 촉촉하다, 쫀득, 최고, 이다, 값해, ..."
1,올리브영,베이스메이크업,"[스킨, 케어, 꼭, 해, 아토, 베리, 라인, 순하다, 추천, 합, 회사, 우선,..."
2,올리브영,클렌징,"[여드름, 피부, 진정, 기능, 초록색, 다른, 알칼리성, 안제, 자극, 심하다, ..."
3,올리브영,마스크팩,"[데카, 사이드, 흔적, 리페, 가성, 좋다, 데카, 사이드, 성분, 피부, 장벽,..."
4,올리브영,선케어,"[달바, 담당자, 재활용, 가능하다, 용기, 의, 변경, 검토, 부탁드리다, 미스트..."
5,올리브영,립메이크업,"[코지, 위스퍼, 살짝, 따뜻하다, 브라운, 베이지, 색상, 차분하다, 우아하다, ..."
6,올리브영,아이메이크업,"[왜, 기존, 트, 라떼, 단종, 에스쁘, 사실, 브랜드, 리뉴, 단종, 부활, 말..."


## 단어 빈도수 계산

In [ ]:
# 5. 단어 빈도수 계산
# 스킨케어

counts = Counter(olive_top5_df['bucket_list'][0])
test_counts = counts.most_common()
ret_0 = dict((x, y) for x,y in test_counts)

ret_0_df = pd.DataFrame.from_dict(ret_0, orient='index')
ret_0_df.reset_index(inplace=True)

ret_0_df.columns = ['Word', 'Frequency']
ret_0_df['플랫폼'], ret_0_df['중분류1'] = ['올리브영', '스킨케어']

ret_0_df = ret_0_df.reindex(['플랫폼', '중분류1', 'Word', 'Frequency'], axis=1)

ret_0_df.head()

,플랫폼,중분류1,Word,Frequency
0,올리브영,스킨케어,좋다,891
1,올리브영,스킨케어,피부,528
2,올리브영,스킨케어,크림,323
3,올리브영,스킨케어,보습,264
4,올리브영,스킨케어,촉촉하다,172


In [ ]:
# 5. 단어 빈도수 계산
# 베이스메이크업

counts = Counter(olive_top5_df['bucket_list'][1])
test_counts = counts.most_common()
ret_1 = dict((x, y) for x,y in test_counts)

ret_1_df = pd.DataFrame.from_dict(ret_1, orient='index')
ret_1_df.reset_index(inplace=True)

ret_1_df.columns = ['Word', 'Frequency']
ret_1_df['플랫폼'], ret_1_df['중분류1'] = ['올리브영', '베이스메이크업']

ret_1_df = ret_1_df.reindex(['플랫폼', '중분류1', 'Word', 'Frequency'], axis=1)

ret_1_df.head()

,플랫폼,중분류1,Word,Frequency
0,올리브영,베이스메이크업,좋다,666
1,올리브영,베이스메이크업,피부,305
2,올리브영,베이스메이크업,미스트,263
3,올리브영,베이스메이크업,분사,211
4,올리브영,베이스메이크업,건조하다,157


In [ ]:
# 5. 단어 빈도수 계산
# 클렌징 

counts = Counter(olive_top5_df['bucket_list'][2])
test_counts = counts.most_common()
ret_2 = dict((x, y) for x,y in test_counts)

ret_2_df = pd.DataFrame.from_dict(ret_2, orient='index')
ret_2_df.reset_index(inplace=True)

ret_2_df.columns = ['Word', 'Frequency']
ret_2_df['플랫폼'], ret_2_df['중분류1'] = ['올리브영', '클렌징']

ret_2_df = ret_2_df.reindex(['플랫폼', '중분류1', 'Word', 'Frequency'], axis=1)

ret_2_df.head()

,플랫폼,중분류1,Word,Frequency
0,올리브영,클렌징,좋다,815
1,올리브영,클렌징,피부,421
2,올리브영,클렌징,자극,270
3,올리브영,클렌징,세안,238
4,올리브영,클렌징,피지,228


In [ ]:
# 5. 단어 빈도수 계산
# 마스크팩  

counts = Counter(olive_top5_df['bucket_list'][3])
test_counts = counts.most_common()
ret_3 = dict((x, y) for x,y in test_counts)

ret_3_df = pd.DataFrame.from_dict(ret_3, orient='index')
ret_3_df.reset_index(inplace=True)

ret_3_df.columns = ['Word', 'Frequency']
ret_3_df['플랫폼'], ret_3_df['중분류1'] = ['올리브영', '마스크팩']

ret_3_df = ret_3_df.reindex(['플랫폼', '중분류1', 'Word', 'Frequency'], axis=1)

ret_3_df.head()

,플랫폼,중분류1,Word,Frequency
0,올리브영,마스크팩,좋다,658
1,올리브영,마스크팩,피부,428
2,올리브영,마스크팩,팩,308
3,올리브영,마스크팩,마스크,158
4,올리브영,마스크팩,패치,142


In [ ]:
# 5. 단어 빈도수 계산
# 선케어

counts = Counter(olive_top5_df['bucket_list'][4])
test_counts = counts.most_common()
ret_4 = dict((x, y) for x,y in test_counts)

ret_4_df = pd.DataFrame.from_dict(ret_4, orient='index')
ret_4_df.reset_index(inplace=True)

ret_4_df.columns = ['Word', 'Frequency']
ret_4_df['플랫폼'], ret_4_df['중분류1'] = ['올리브영', '선케어']

ret_4_df = ret_4_df.reindex(['플랫폼', '중분류1', 'Word', 'Frequency'], axis=1)

ret_4_df.head()

,플랫폼,중분류1,Word,Frequency
0,올리브영,선케어,선크림,802
1,올리브영,선케어,좋다,765
2,올리브영,선케어,피부,522
3,올리브영,선케어,톤업,280
4,올리브영,선케어,촉촉하다,236


In [ ]:
# 5. 단어 빈도수 계산
# 립메이크업

counts = Counter(olive_top5_df['bucket_list'][5])
test_counts = counts.most_common()
ret_5 = dict((x, y) for x,y in test_counts)

ret_5_df = pd.DataFrame.from_dict(ret_5, orient='index')
ret_5_df.reset_index(inplace=True)

ret_5_df.columns = ['Word', 'Frequency']
ret_5_df['플랫폼'], ret_5_df['중분류1'] = ['올리브영', '립메이크업']

ret_5_df = ret_5_df.reindex(['플랫폼', '중분류1', 'Word', 'Frequency'], axis=1)

ret_5_df.head()

,플랫폼,중분류1,Word,Frequency
0,올리브영,립메이크업,좋다,752
1,올리브영,립메이크업,컬러,652
2,올리브영,립메이크업,입술,448
3,올리브영,립메이크업,립,426
4,올리브영,립메이크업,틴트,394


In [ ]:
# 5. 단어 빈도수 계산
# 립메이크업

counts = Counter(olive_top5_df['bucket_list'][6])
test_counts = counts.most_common()
ret_6 = dict((x, y) for x,y in test_counts)

ret_6_df = pd.DataFrame.from_dict(ret_6, orient='index')
ret_6_df.reset_index(inplace=True)

ret_6_df.columns = ['Word', 'Frequency']
ret_6_df['플랫폼'], ret_6_df['중분류1'] = ['올리브영', '아이메이크업']

ret_6_df = ret_6_df.reindex(['플랫폼', '중분류1', 'Word', 'Frequency'], axis=1)

ret_6_df.head()

,플랫폼,중분류1,Word,Frequency
0,올리브영,아이메이크업,좋다,668
1,올리브영,아이메이크업,예쁘다,261
2,올리브영,아이메이크업,컬러,234
3,올리브영,아이메이크업,마스카라,233
4,올리브영,아이메이크업,지속,206


In [ ]:
# 6. 빈도수 데이터셋 통합
concat_list = [ret_0_df, ret_1_df, ret_2_df, ret_3_df, ret_4_df, ret_5_df, ret_6_df]
olive_ret_df = pd.concat(concat_list, ignore_index=True)

print(olive_ret_df['중분류1'].unique())
print(olive_ret_df.info())
print(olive_ret_df.head())

['스킨케어' '베이스메이크업' '클렌징' '마스크팩' '선케어' '립메이크업' '아이메이크업']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16029 entries, 0 to 16028
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   플랫폼        16029 non-null  object
 1   중분류1       16029 non-null  object
 2   Word       16029 non-null  object
 3   Frequency  16029 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 501.0+ KB
None
    플랫폼  중분류1  Word  Frequency
0  올리브영  스킨케어    좋다        891
1  올리브영  스킨케어    피부        528
2  올리브영  스킨케어    크림        323
3  올리브영  스킨케어    보습        264
4  올리브영  스킨케어  촉촉하다        172


In [ ]:
# 데이터셋 저장
# olive_ret_df.to_csv('올리브영_워드클라우드.csv', index=False, encoding="utf-8-sig")